In [18]:
#rain precipitation from TRMM/GPM has been normalized into HDF5 by IMERG
#example
#wget https://arthurhouhttps.pps.eosdis.nasa.gov/gpmallversions/V06/2012/10/21/imerg/3B-HHR.MS.MRG.3IMERG.20121021-S170000-E172959.1020.V06B.HDF5
#

from ipynb.fs.full.utils import *
from glob import glob
import os
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import csv
import h5py
import numpy as np
diameter = 5



In [21]:
def date_to_nth_day(date, fmt='%Y%m%d'):
    date = datetime.datetime.strptime(date, fmt)
    new_year_day = datetime.datetime(year=date.year, month=1, day=1)
    return (date - new_year_day).days + 1

def parse_html(html_file):
    '''
    parse html to get file list
    '''       
    with open(html_file, 'r') as input:
        soup = BeautifulSoup(input, "html.parser").find_all(lambda t: t.name == "a" and t.text.startswith('3B'))
        filelist = []
        for it in soup:
            filelist.append(it["href"])
        return filelist
    

def download_filelist(folder, url):
    print ('downloading to ', folder)
    
    username = 'xiaogang.liu1@gmail.com'
    password = 'xiaogang.liu1@gmail.com'

    filename = folder + 'index.html'
    
    r = requests.get(url, auth = (username, password))

    if r.status_code == 200:
        print ('writing to', filename)
        with open(filename, 'wb') as out:
            for bits in r.iter_content():
                out.write(bits)
                
        file_list = parse_html(filename)
        
        return file_list
        
def download_file(folder, url, filename):
     
    username = 'xiaogang.liu1@gmail.com'
    password = 'xiaogang.liu1@gmail.com'
    
    print ('downloading file ', url + filename)
    
    r = requests.get(url + filename, auth = (username, password))
    if r.status_code == 200:
        print ('writing to', folder + filename)
        with open(folder + filename, 'wb') as out:
            for bits in r.iter_content():
                out.write(bits)
    else:
        print ('download error ', r.status_code)
    
     
def generate_imerg_url(datestr):
    '''
    compose url using date  'YYYYMMDD'
    '''
    #url = 'https://arthurhouhttps.pps.eosdis.nasa.gov/gpmallversions/V06/' + datestr[0:4] + '/' + datestr[4:6] + '/' + datestr[6:8] + '/imerg/'
    
    url = 'https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/'+ datestr[0:4] + '/' + str(date_to_nth_day(datestr)) + '/'
    
    print (url)
    return url

    
def get_target_file(filelist, timestr):
    '''
    return file that matches timestamp
    '''
    key = 'S' + timestr
    print (key)
    for it in filelist:
        if key in it:
            return it
    return None

    
def download_imerg(folder):
    json_profile = folder + '/profile.json'
    json_file = open(json_profile)
    d = json.load(json_file)
        
    for it in d['path']:
        print(it)
        
        url = generate_imerg_url(str(it['date']))
        
        filelist = download_filelist(folder, url)
        
        if filelist is None:
            continue
        
        filename = get_target_file(filelist, str(it['time']))  # without path
        if filename is None:
            continue
        
        download_file(folder, url, filename)
        
        #this is the downloaded file
        full_path_filename = folder + filename
        post_processing(it, folder, full_path_filename, it['lg'], it['lt'])

def post_processing(param, folder, full_path_filename, lonstr, latstr): # lon and lat should be read from the path name or given externally
    '''
        1) pick the data only in bounding  box
        2) output to CSV with date_time
        3) remove the downloaded hdf5 files
    '''    
    lon = 0
    lat = 0
    sign = -1.0 if 'W' in lonstr else 1.0
    lon = sign * float(lonstr[:-1])
    sign = -1.0 if 'S' in latstr else 1.0
    lat = sign * float(latstr[:-1])
    
    hdf_array = h5py.File(full_path_filename, 'r')
    group = hdf_array['Grid']
    
    longitude = group['lon'][()] # len 3600, x-axis
    latitude = group['lat'][()] # len 1800, y-axis
    rain = group['precipitationCal'][()] # array of shape (1, 3600, 1800)
            
    x = int((lon-longitude[0])*10)
    y = int((lat-latitude[0])*10)
    
    
    data = hdf_array['Grid']['precipitationCal'][:, x-50:x+50, y-50:y+50] # 100 by 100, one hundred square degrees (#1)

    datestr = str(param['date'])
    timestr = str(param['time'])
    final = np.array([data])

    pathname = folder + 'imerg_precipitation_' + datestr + '_' + timestr + '.npy'
    np.save(pathname, final) # write np array to .npy file with pre-existing file name + datestr + timestr (#2)

    os.remove(full_path_filename) # remove previous .HDF5 file (#3)
  

In [23]:
#getDayPosition('/home/fun/data/AL182012/profile.json')

hurricanes_folders = glob("/home/fun/data/AL*2012/", recursive = True)

for it in hurricanes_folders:
    download_imerg(it)



{'date': 20120519, 'time': '0000', 'category': 'LO', 'lt': '33.1N', 'lg': '77.0W', 'maxwind': '25', 'maxpres': '1010', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/140/
downloading to  /home/fun/data/AL012012/
writing to /home/fun/data/AL012012/index.html
S0000
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/140/3B-HHR.MS.MRG.3IMERG.20120519-S000000-E002959.0000.V06B.HDF5
writing to /home/fun/data/AL012012/3B-HHR.MS.MRG.3IMERG.20120519-S000000-E002959.0000.V06B.HDF5
{'date': 20120519, 'time': '0600', 'category': 'TD', 'lt': '32.8N', 'lg': '77.1W', 'maxwind': '30', 'maxpres': '1008', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse'

writing to /home/fun/data/AL012012/3B-HHR.MS.MRG.3IMERG.20120521-S180000-E182959.1080.V06B.HDF5
{'date': 20120522, 'time': '0000', 'category': 'TS', 'lt': '30.7N', 'lg': '77.1W', 'maxwind': '35', 'maxpres': '1007', '34ktne': '40', '34ktse': '30', '34ktsw': '30', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/143/
downloading to  /home/fun/data/AL012012/
writing to /home/fun/data/AL012012/index.html
S0000
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/143/3B-HHR.MS.MRG.3IMERG.20120522-S000000-E002959.0000.V06B.HDF5
writing to /home/fun/data/AL012012/3B-HHR.MS.MRG.3IMERG.20120522-S000000-E002959.0000.V06B.HDF5
{'date': 20120522, 'time': '0600', 'category': 'TS', 'lt': '31.5N', 'lg': '76.1W', 'maxwind': '35', 'maxpres': '1007', '34ktne': '40', '34ktse': '30', '34ktsw': '3

{'date': 20120526, 'time': '0600', 'category': 'SS', 'lt': '32.4N', 'lg': '75.3W', 'maxwind': '40', 'maxpres': '1001', '34ktne': '100', '34ktse': '60', '34ktsw': '60', '34ktnw': '40', '50ktne': '100', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/147/
downloading to  /home/fun/data/AL022012/
writing to /home/fun/data/AL022012/index.html
S0600
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/147/3B-HHR.MS.MRG.3IMERG.20120526-S060000-E062959.0360.V06B.HDF5
writing to /home/fun/data/AL022012/3B-HHR.MS.MRG.3IMERG.20120526-S060000-E062959.0360.V06B.HDF5
{'date': 20120526, 'time': '1200', 'category': 'SS', 'lt': '32.0N', 'lg': '75.9W', 'maxwind': '40', 'maxpres': '1000', '34ktne': '100', '34ktse': '60', '34ktsw': '60', '34ktnw': '40', '50ktne': '100', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne':

writing to /home/fun/data/AL022012/3B-HHR.MS.MRG.3IMERG.20120528-S120000-E122959.0720.V06B.HDF5
{'date': 20120528, 'time': '1800', 'category': 'TD', 'lt': '30.5N', 'lg': '83.0W', 'maxwind': '30', 'maxpres': '1003', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/149/
downloading to  /home/fun/data/AL022012/
writing to /home/fun/data/AL022012/index.html
S1800
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/149/3B-HHR.MS.MRG.3IMERG.20120528-S180000-E182959.1080.V06B.HDF5
writing to /home/fun/data/AL022012/3B-HHR.MS.MRG.3IMERG.20120528-S180000-E182959.1080.V06B.HDF5
{'date': 20120529, 'time': '0000', 'category': 'TD', 'lt': '30.6N', 'lg': '83.2W', 'maxwind': '25', 'maxpres': '1005', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '3

{'date': 20120531, 'time': '1200', 'category': 'LO', 'lt': '37.1N', 'lg': '70.4W', 'maxwind': '35', 'maxpres': '998', '34ktne': '0', '34ktse': '130', '34ktsw': '130', '34ktnw': '100', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/152/
downloading to  /home/fun/data/AL022012/
writing to /home/fun/data/AL022012/index.html
S1200
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/152/3B-HHR.MS.MRG.3IMERG.20120531-S120000-E122959.0720.V06B.HDF5
writing to /home/fun/data/AL022012/3B-HHR.MS.MRG.3IMERG.20120531-S120000-E122959.0720.V06B.HDF5
{'date': 20120531, 'time': '1800', 'category': 'LO', 'lt': '37.8N', 'lg': '67.6W', 'maxwind': '35', 'maxpres': '1000', '34ktne': '0', '34ktse': '130', '34ktsw': '130', '34ktnw': '100', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0

...

writing to /home/fun/data/AL182012/3B-HHR.MS.MRG.3IMERG.20121023-S180000-E182959.1080.V06B.HDF5
{'date': 20121024, 'time': '0000', 'category': 'TS', 'lt': '14.7N', 'lg': '77.3W', 'maxwind': '55', 'maxpres': '990', '34ktne': '100', '34ktse': '150', '34ktsw': '150', '34ktnw': '40', '50ktne': '40', '50ktse': '0', '50ktsw': '70', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/298/
downloading to  /home/fun/data/AL182012/
writing to /home/fun/data/AL182012/index.html
S0000
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/298/3B-HHR.MS.MRG.3IMERG.20121024-S000000-E002959.0000.V06B.HDF5
writing to /home/fun/data/AL182012/3B-HHR.MS.MRG.3IMERG.20121024-S000000-E002959.0000.V06B.HDF5
{'date': 20121024, 'time': '0600', 'category': 'TS', 'lt': '15.6N', 'lg': '77.1W', 'maxwind': '60', 'maxpres': '987', '34ktne': '100', '34ktse': '150', '34kts

writing to /home/fun/data/AL182012/3B-HHR.MS.MRG.3IMERG.20121026-S000000-E002959.0000.V06B.HDF5
{'date': 20121026, 'time': '0600', 'category': 'HU', 'lt': '25.7N', 'lg': '76.4W', 'maxwind': '70', 'maxpres': '968', '34ktne': '300', '34ktse': '300', '34ktsw': '300', '34ktnw': '160', '50ktne': '240', '50ktse': '90', '50ktsw': '70', '50ktnw': '50', '64ktne': '70', '64ktse': '30', '64ktsw': '20', '64ktnw': '0', 'maxwindradius': '60'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/300/
downloading to  /home/fun/data/AL182012/
writing to /home/fun/data/AL182012/index.html
S0600
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/300/3B-HHR.MS.MRG.3IMERG.20121026-S060000-E062959.0360.V06B.HDF5
writing to /home/fun/data/AL182012/3B-HHR.MS.MRG.3IMERG.20121026-S060000-E062959.0360.V06B.HDF5
{'date': 20121026, 'time': '1200', 'category': 'HU', 'lt': '26.4N', 'lg': '76.9W', 'maxwind': '65', 'maxpres': '970', '34ktne': '360', '34ktse': '240'

writing to /home/fun/data/AL182012/3B-HHR.MS.MRG.3IMERG.20121028-S180000-E182959.1080.V06B.HDF5
{'date': 20121029, 'time': '0000', 'category': 'HU', 'lt': '33.9N', 'lg': '71.0W', 'maxwind': '70', 'maxpres': '950', '34ktne': '450', '34ktse': '300', '34ktsw': '300', '34ktnw': '400', '50ktne': '270', '50ktse': '150', '50ktsw': '150', '50ktnw': '200', '64ktne': '200', '64ktse': '0', '64ktsw': '0', '64ktnw': '150', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/303/
downloading to  /home/fun/data/AL182012/
writing to /home/fun/data/AL182012/index.html
S0000
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/303/3B-HHR.MS.MRG.3IMERG.20121029-S000000-E002959.0000.V06B.HDF5
writing to /home/fun/data/AL182012/3B-HHR.MS.MRG.3IMERG.20121029-S000000-E002959.0000.V06B.HDF5
{'date': 20121029, 'time': '0600', 'category': 'HU', 'lt': '35.3N', 'lg': '70.5W', 'maxwind': '80', 'maxpres': '947', '34ktne': '420', '34ktse': '3

writing to /home/fun/data/AL182012/3B-HHR.MS.MRG.3IMERG.20121031-S000000-E002959.0000.V06B.HDF5
{'date': 20121031, 'time': '0600', 'category': 'EX', 'lt': '41.1N', 'lg': '80.3W', 'maxwind': '35', 'maxpres': '993', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '34ktnw': '0', '50ktne': '380', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/305/
downloading to  /home/fun/data/AL182012/
writing to /home/fun/data/AL182012/index.html
S0600
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/305/3B-HHR.MS.MRG.3IMERG.20121031-S060000-E062959.0360.V06B.HDF5
writing to /home/fun/data/AL182012/3B-HHR.MS.MRG.3IMERG.20121031-S060000-E062959.0360.V06B.HDF5
{'date': 20121031, 'time': '1200', 'category': 'EX', 'lt': '41.5N', 'lg': '80.7W', 'maxwind': '30', 'maxpres': '995', '34ktne': '0', '34ktse': '0', '34ktsw': '0', '3

{'date': 20121024, 'time': '0000', 'category': 'TS', 'lt': '26.5N', 'lg': '49.6W', 'maxwind': '35', 'maxpres': '1004', '34ktne': '40', '34ktse': '40', '34ktsw': '40', '34ktnw': '0', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '64ktse': '0', '64ktsw': '0', '64ktnw': '0', 'maxwindradius': '0'}
https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/298/
downloading to  /home/fun/data/AL192012/
writing to /home/fun/data/AL192012/index.html
S0000
downloading file  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.06/2012/298/3B-HHR.MS.MRG.3IMERG.20121024-S000000-E002959.0000.V06B.HDF5
writing to /home/fun/data/AL192012/3B-HHR.MS.MRG.3IMERG.20121024-S000000-E002959.0000.V06B.HDF5
{'date': 20121024, 'time': '0600', 'category': 'TS', 'lt': '27.2N', 'lg': '48.5W', 'maxwind': '40', 'maxpres': '1003', '34ktne': '50', '34ktse': '60', '34ktsw': '60', '34ktnw': '40', '50ktne': '0', '50ktse': '0', '50ktsw': '0', '50ktnw': '0', '64ktne': '0', '

In [15]:
#test hdf5 imerg

#imerg_df = pd.read_hdf('/home/fun/data/imerg/3B-HHR.MS.MRG.3IMERG.20121021-S023000-E025959.0150.V06B.HDF5')  

import h5py

f = h5py.File('/home/fun/data/AL022004/3B-HHR.MS.MRG.3IMERG.20040803-S180000-E182959.1080.V06B.HDF5', 'r')

for key in f.keys():
    print(key) #Names of the root level object names in HDF5 file - can be groups or datasets.
    print(type(f[key])) # get the object type: usually group or dataset
    
#Get the HDF5 group; key needs to be a group name from above
group = f['Grid']

#Checkout what keys are inside that group.
for key in group.keys():
    print(key)
    
longitude = group['lon'][()] # len 3600, x-axis
latitude = group['lat'][()] # len 1800, y-axis
rain = group['precipitationCal'][()] # array of shape (1, 3600, 1800)

lon = -77.4
lat = 14.3

#print(latitude, clos_coords)
i = int((lat-latitude[0])*10)
j = int((lon-longitude[0])*10)
print(latitude[i], longitude[j], i, j)

rain = rain.reshape(3600, 1800)

sliced = rain[j-5:j+5, i-5:i+5]
sliced.shape

OSError: Unable to open file (unable to open file: name = '/home/fun/data/AL022004/3B-HHR.MS.MRG.3IMERG.20040803-S180000-E182959.1080.V06B.HDF5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [12]:
import numpy as np
filename = '/home/fun/data/AL022004/3B-HHR.MS.MRG.3IMERG.20040803-S180000-E182959.1080.V06B.HDF5'

lon = -77.4
lat = 14.3

#print(latitude, clos_coords)
x = int((lon-longitude[0])*10)
y = int((lat-latitude[0])*10)

hdf_array = h5py.File(filename, 'r')
data = hdf_array['Grid']['precipitationCal'][:, x-5:x+5, y-5:y+5]

data

NameError: name 'longitude' is not defined

In [85]:
file = '/home/fun/data/AL022004/3B-HHR.MS.MRG.3IMERG.20121021-S000000-E002959.0000.V06B.HDF5'
len(file)

name = file[:len(file)-5]
name

'/home/fun/data/AL022004/3B-HHR.MS.MRG.3IMERG.20121021-S000000-E002959.0000.V06B'

In [179]:
array = np.load('/home/fun/data/AL022004/imerg_precipitation_20040803_1200.npy')
array

array([[[[ 0.06247815,  0.29016453,  1.0588278 ,  2.2504165 ,
           2.343059  ,  2.4157019 , 10.181723  , 12.900776  ,
          26.127796  , 30.346048  ],
         [ 0.37743443,  0.28462726,  0.28016454,  2.1891775 ,
           2.1970284 ,  9.379759  ,  8.414813  , 19.552551  ,
          24.830242  , 23.657272  ],
         [ 0.39421082,  0.9118869 ,  1.2785861 ,  2.6486425 ,
           2.723851  ,  5.715255  , 10.913759  , 25.128246  ,
          24.038334  , 25.0345    ],
         [ 1.0279177 ,  0.93866324,  1.2685862 ,  1.2039589 ,
           2.5193882 ,  4.1288385 ,  5.5526223 ,  6.9541235 ,
          11.752715  , 21.643759  ],
         [ 1.1708999 ,  1.1753626 ,  1.3354396 ,  1.3797377 ,
           1.9213829 ,  4.1697483 ,  5.4864163 ,  7.268257  ,
           9.392678  , 11.334448  ],
         [ 1.6964267 ,  1.4473368 ,  1.3636196 ,  1.3289921 ,
           1.7290797 ,  2.1481695 ,  2.4771826 ,  2.4582572 ,
           2.5453625 ,  6.917589  ],
         [ 3.4169512 ,  2.0160208 

NameError: name 'latitude' is not defined